# DAY 1 - NOTEBOOK 2: TRANSFORMATIONS vs ACTIONS & LAZY EVALUATION

---

## 🎯 Learning Objectives:
1. Understand Transformations vs Actions
2. Master Lazy Evaluation concept
3. Learn Narrow vs Wide Transformations
4. Read and understand Execution Plans
5. Optimize query performance

---

## SETUP: CREATE SPARK SESSION

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col, count, avg, min, max
from pyspark.sql.types import *
import time

# Stop existing session
try:
    spark.stop()
    print("✅ Stopped existing session")
    time.sleep(2)
except:
    pass

# Create new session
spark = SparkSession.builder \
    .appName("Day1-TransformationsActions") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "2") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

print("✅ Spark Session Created")
print(f"   App ID: {spark.sparkContext.applicationId}")
print(f"   UI: http://localhost:4040")

✅ Stopped existing session
✅ Spark Session Created
   App ID: app-20251222155214-0002
   UI: http://localhost:4040


25/12/22 15:52:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


---

## PART 1: TRANSFORMATIONS vs ACTIONS

### 📚 Theory:

```
┌──────────────────────────────────────────────────────────────┐
│                  TRANSFORMATIONS                             │
├──────────────────────────────────────────────────────────────┤
│  • LAZY (không execute ngay)                                 │
│  • Return DataFrame mới                                      │
│  • Build execution plan                                      │
│  • Không trigger computation                                 │
│                                                              │
│  Examples:                                                   │
│    - select()      - filter()       - withColumn()          │
│    - groupBy()     - join()         - orderBy()             │
│    - distinct()    - drop()         - union()               │
└──────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────┐
│                     ACTIONS                                  │
├──────────────────────────────────────────────────────────────┤
│  • EAGER (execute ngay lập tức)                              │
│  • Trigger computation                                       │
│  • Return results to driver                                  │
│  • Thực thi tất cả transformations trước đó                  │
│                                                              │
│  Examples:                                                   │
│    - show()        - count()        - collect()             │
│    - take()        - first()        - write()               │
│    - head()        - foreach()      - reduce()              │
└──────────────────────────────────────────────────────────────┘
```

### 🔬 Demo: Lazy Evaluation

In [34]:
# Create sample data
data = [(i, f"Name_{i}", i * 10) for i in range(1, 101)]
df = spark.createDataFrame(data, ["id", "name", "value"])

print("✅ Created DataFrame with 100 rows")
print(f"   Partitions: {df.rdd.getNumPartitions()}")

✅ Created DataFrame with 100 rows
   Partitions: 2


In [35]:
print("📌 Step 1: Define transformations (LAZY - No execution yet)")
print("⏱️  Starting...")
start = time.time()

# These are transformations - NOT executed yet!
df_filtered = df.filter(col("value") > 50)
df_selected = df_filtered.select("id", "name")
df_renamed = df_selected.withColumnRenamed("name", "employee_name")

elapsed = time.time() - start
print(f"✅ Transformations defined in {elapsed:.4f}s")
print("   ⚠️  No actual computation happened yet!")
print("   ⚠️  Data was NOT processed!")
print("   ⚠️  Only execution plan was created!")

📌 Step 1: Define transformations (LAZY - No execution yet)
⏱️  Starting...
✅ Transformations defined in 0.0338s
   ⚠️  No actual computation happened yet!
   ⚠️  Data was NOT processed!
   ⚠️  Only execution plan was created!


In [36]:
print("📌 Step 2: Trigger action (EAGER - Execution happens now!)")
print("⏱️  Executing...")
start = time.time()

# This is an action - triggers execution of ALL transformations above
result_count = df_renamed.count()

elapsed = time.time() - start
print(f"✅ Action completed in {elapsed:.4f}s")
print(f"   Result: {result_count} rows")
print("   ✅ NOW all transformations were executed!")
print("   ✅ Data was actually processed!")

📌 Step 2: Trigger action (EAGER - Execution happens now!)
⏱️  Executing...


✅ Action completed in 2.9687s
   Result: 95 rows
   ✅ NOW all transformations were executed!
   ✅ Data was actually processed!


### 💡 Key Insight:

```
Transformations:  0.0001s (instant) → Just planning
Action:           0.5s (slow)       → Actual execution

Why?
• Transformations chỉ build execution plan
• Actions trigger thực thi plan đó
• Spark optimize toàn bộ plan trước khi execute
```

---

## PART 2: EXECUTION PLANS

In [37]:
print("📌 Logical Plan (What to do)")
df_renamed.explain(extended=False)

📌 Logical Plan (What to do)
== Physical Plan ==
*(1) Project [id#608L, name#609 AS employee_name#616]
+- *(1) Filter (isnotnull(value#610L) AND (value#610L > 50))
   +- *(1) Scan ExistingRDD[id#608L,name#609,value#610L]




In [38]:
print("📌 Physical Plan (How to do)")
df_renamed.explain(mode="formatted")

📌 Physical Plan (How to do)
== Physical Plan ==
* Project (3)
+- * Filter (2)
   +- * Scan ExistingRDD (1)


(1) Scan ExistingRDD [codegen id : 1]
Output [3]: [id#608L, name#609, value#610L]
Arguments: [id#608L, name#609, value#610L], MapPartitionsRDD[4] at applySchemaToPythonRDD at <unknown>:0, ExistingRDD, UnknownPartitioning(0)

(2) Filter [codegen id : 1]
Input [3]: [id#608L, name#609, value#610L]
Condition : (isnotnull(value#610L) AND (value#610L > 50))

(3) Project [codegen id : 1]
Output [2]: [id#608L, name#609 AS employee_name#616]
Input [3]: [id#608L, name#609, value#610L]




### 📖 Reading Execution Plans:

```
Execution plan đọc từ DƯỚI LÊN TRÊN:

== Physical Plan ==
Project [id, name AS employee_name]     ← Step 3: Rename column
+- Project [id, name]                   ← Step 2: Select columns
   +- Filter (value > 50)               ← Step 1: Filter rows
      +- Scan                           ← Step 0: Read data

Spark đọc từ dưới lên và optimize:
1. Scan data
2. Filter ngay (predicate pushdown)
3. Select only needed columns (column pruning)
4. Rename
```

---

## PART 3: NARROW vs WIDE TRANSFORMATIONS

### 📚 Theory:

```
┌──────────────────────────────────────────────────────────────┐
│              NARROW TRANSFORMATIONS                          │
├──────────────────────────────────────────────────────────────┤
│  • Không cần shuffle data                                    │
│  • Mỗi partition xử lý độc lập                               │
│  • Nhanh, hiệu quả                                           │
│  • 1 input partition → 1 output partition                    │
│                                                              │
│  Examples:                                                   │
│    - select()      - filter()       - withColumn()          │
│    - map()         - flatMap()      - union()               │
│                                                              │
│  Diagram:                                                    │
│    Partition 1 → [filter] → Partition 1'                    │
│    Partition 2 → [filter] → Partition 2'                    │
│    Partition 3 → [filter] → Partition 3'                    │
│    (No data movement between partitions)                     │
└──────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────┐
│               WIDE TRANSFORMATIONS                           │
├──────────────────────────────────────────────────────────────┤
│  • Cần shuffle data giữa partitions                          │
│  • Network I/O intensive                                     │
│  • Chậm hơn narrow transformations                           │
│  • N input partitions → M output partitions                  │
│                                                              │
│  Examples:                                                   │
│    - groupBy()     - join()         - orderBy()             │
│    - distinct()    - repartition()  - coalesce()            │
│                                                              │
│  Diagram:                                                    │
│    Partition 1 ──┐                                          │
│    Partition 2 ──┼→ [shuffle] → Partition 1'                │
│    Partition 3 ──┘              → Partition 2'              │
│    (Data moves across network)                               │
└──────────────────────────────────────────────────────────────┘
```

### 🔬 Demo: Narrow Transformation

In [39]:
print("📌 Narrow Transformation: filter + select")
print("   (No shuffle, fast)")

start = time.time()
df_narrow = df.filter(col("value") > 50).select("id", "name")
result = df_narrow.count()
elapsed = time.time() - start

print(f"✅ Result: {result} rows")
print(f"⚡ Time: {elapsed:.4f}s")
print("\n📊 Sample:")
df_narrow.show(5)

📌 Narrow Transformation: filter + select
   (No shuffle, fast)
✅ Result: 95 rows
⚡ Time: 0.5208s

📊 Sample:
+---+-------+
| id|   name|
+---+-------+
|  6| Name_6|
|  7| Name_7|
|  8| Name_8|
|  9| Name_9|
| 10|Name_10|
+---+-------+
only showing top 5 rows



### 🔬 Demo: Wide Transformation

In [40]:
print("📌 Wide Transformation: groupBy + agg")
print("   (Requires shuffle, slower)")

start = time.time()
df_wide = df.groupBy("id").agg(sum("value").alias("total"))
result = df_wide.count()
elapsed = time.time() - start

print(f"✅ Result: {result} rows")
print(f"⚡ Time: {elapsed:.4f}s")
print("   ⚠️  Notice: Slower than narrow transformation!")
print("\n📊 Sample:")
df_wide.show(5)

📌 Wide Transformation: groupBy + agg
   (Requires shuffle, slower)
✅ Result: 100 rows
⚡ Time: 0.9512s
   ⚠️  Notice: Slower than narrow transformation!

📊 Sample:
+---+-----+
| id|total|
+---+-----+
|  2|   20|
|  4|   40|
|  5|   50|
|  8|   80|
| 12|  120|
+---+-----+
only showing top 5 rows



### 📊 Performance Comparison

In [41]:
# Create larger dataset for better comparison
large_data = [(i, f"Name_{i}", i * 10) for i in range(1, 10001)]
df_large = spark.createDataFrame(large_data, ["id", "name", "value"])

print(f"✅ Created dataset with {df_large.count()} rows")
print(f"   Partitions: {df_large.rdd.getNumPartitions()}")

✅ Created dataset with 10000 rows
   Partitions: 2


In [42]:
print("🏁 Performance Test: Narrow vs Wide")
print("="*60)

# Test 1: Narrow transformation
print("\n📌 Test 1: Narrow (filter + select)")
start = time.time()
result_narrow = df_large.filter(col("value") > 5000).select("id", "name").count()
time_narrow = time.time() - start
print(f"   Result: {result_narrow} rows")
print(f"   Time: {time_narrow:.4f}s")

# Test 2: Wide transformation
print("\n📌 Test 2: Wide (groupBy + agg)")
start = time.time()
result_wide = df_large.groupBy("id").agg(sum("value").alias("total")).count()
time_wide = time.time() - start
print(f"   Result: {result_wide} rows")
print(f"   Time: {time_wide:.4f}s")

# Comparison
print("\n" + "="*60)
print("📊 COMPARISON:")
print(f"   Narrow: {time_narrow:.4f}s")
print(f"   Wide:   {time_wide:.4f}s")
print(f"   Wide is {time_wide/time_narrow:.2f}x slower")
print("\n💡 Why? Wide transformation requires shuffle (network I/O)")

🏁 Performance Test: Narrow vs Wide

📌 Test 1: Narrow (filter + select)
   Result: 9500 rows
   Time: 0.2735s

📌 Test 2: Wide (groupBy + agg)
   Result: 10000 rows
   Time: 0.4837s

📊 COMPARISON:
   Narrow: 0.2735s
   Wide:   0.4837s
   Wide is 1.77x slower

💡 Why? Wide transformation requires shuffle (network I/O)


---

## PART 4: COMMON ACTIONS

In [43]:
# Sample DataFrame
sample_df = df.limit(10)

print("📌 Common Actions Demo")
print("="*60)

📌 Common Actions Demo


In [44]:
print("\n1️⃣ count() - Count rows")
count = sample_df.count()
print(f"   Total rows: {count}")


1️⃣ count() - Count rows
   Total rows: 10


In [45]:
print("\n2️⃣ show() - Display data")
sample_df.show(5)


2️⃣ show() - Display data
+---+------+-----+
| id|  name|value|
+---+------+-----+
|  1|Name_1|   10|
|  2|Name_2|   20|
|  3|Name_3|   30|
|  4|Name_4|   40|
|  5|Name_5|   50|
+---+------+-----+
only showing top 5 rows



In [46]:
print("\n3️⃣ collect() - Get all rows as list")
rows = sample_df.limit(3).collect()
print(f"   Collected {len(rows)} rows")
for row in rows:
    print(f"   {row}")


3️⃣ collect() - Get all rows as list
   Collected 3 rows
   Row(id=1, name='Name_1', value=10)
   Row(id=2, name='Name_2', value=20)
   Row(id=3, name='Name_3', value=30)


In [47]:
print("\n4️⃣ take(n) - Get first n rows")
first_3 = sample_df.take(3)
print(f"   First 3 rows: {first_3}")


4️⃣ take(n) - Get first n rows
   First 3 rows: [Row(id=1, name='Name_1', value=10), Row(id=2, name='Name_2', value=20), Row(id=3, name='Name_3', value=30)]


In [48]:
print("\n5️⃣ first() - Get first row")
first_row = sample_df.first()
print(f"   First row: {first_row}")


5️⃣ first() - Get first row
   First row: Row(id=1, name='Name_1', value=10)


In [49]:
print("\n6️⃣ head(n) - Get first n rows")
head_3 = sample_df.head(3)
print(f"   Head 3: {head_3}")


6️⃣ head(n) - Get first n rows
   Head 3: [Row(id=1, name='Name_1', value=10), Row(id=2, name='Name_2', value=20), Row(id=3, name='Name_3', value=30)]


---

## PART 5: MULTIPLE ACTIONS PROBLEM

### ⚠️ Problem: Multiple actions re-compute everything!

In [50]:
print("📌 Test: Multiple actions WITHOUT cache")
print("   (Each action re-computes from scratch)")
print("="*60)

# Create expensive transformation
df_expensive = df_large.filter(col("value") > 5000)

start = time.time()
count1 = df_expensive.count()  # Action 1: compute
count2 = df_expensive.count()  # Action 2: compute again!
count3 = df_expensive.count()  # Action 3: compute again!
elapsed = time.time() - start

print(f"✅ 3 actions completed")
print(f"   Time: {elapsed:.4f}s")
print("   ⚠️  Each action re-computed the filter!")

📌 Test: Multiple actions WITHOUT cache
   (Each action re-computes from scratch)
✅ 3 actions completed
   Time: 0.5464s
   ⚠️  Each action re-computed the filter!


### ✅ Solution: Use cache()!

In [51]:
print("📌 Test: Multiple actions WITH cache")
print("   (First action computes + caches, rest use cache)")
print("="*60)

# Cache the DataFrame
df_cached = df_large.filter(col("value") > 5000).cache()

start = time.time()
count1 = df_cached.count()  # Action 1: compute + cache
count2 = df_cached.count()  # Action 2: use cache (fast!)
count3 = df_cached.count()  # Action 3: use cache (fast!)
elapsed = time.time() - start

print(f"✅ 3 actions completed")
print(f"   Time: {elapsed:.4f}s")
print("   ⚡ Much faster! Used cache for 2nd and 3rd actions")

# Clean up
df_cached.unpersist()
print("\n✅ Cache cleared")

📌 Test: Multiple actions WITH cache
   (First action computes + caches, rest use cache)
✅ 3 actions completed
   Time: 1.6130s
   ⚡ Much faster! Used cache for 2nd and 3rd actions

✅ Cache cleared


---

## PART 6: CHAINING OPERATIONS

### ❌ Bad Practice: Step by step (verbose)

In [52]:
print("❌ Bad Practice: Step by step")
df1 = df.filter(col("value") > 30)
df2 = df1.select("id", "name", "value")
df3 = df2.withColumn("value_doubled", col("value") * 2)
df4 = df3.orderBy("value", ascending=False)
df4.show(5)

print("\n⚠️  Problems:")
print("   • Too many intermediate variables")
print("   • Hard to read")
print("   • Clutters namespace")

❌ Bad Practice: Step by step
+---+--------+-----+-------------+
| id|    name|value|value_doubled|
+---+--------+-----+-------------+
|100|Name_100| 1000|         2000|
| 99| Name_99|  990|         1980|
| 98| Name_98|  980|         1960|
| 97| Name_97|  970|         1940|
| 96| Name_96|  960|         1920|
+---+--------+-----+-------------+
only showing top 5 rows


⚠️  Problems:
   • Too many intermediate variables
   • Hard to read
   • Clutters namespace


### ✅ Good Practice: Chaining (recommended)

In [53]:
print("✅ Good Practice: Chaining")
result = df \
    .filter(col("value") > 30) \
    .select("id", "name", "value") \
    .withColumn("value_doubled", col("value") * 2) \
    .orderBy("value", ascending=False)

result.show(5)

print("\n✅ Benefits:")
print("   • Clean and readable")
print("   • No intermediate variables")
print("   • Easy to understand flow")
print("   • Professional code style")

✅ Good Practice: Chaining
+---+--------+-----+-------------+
| id|    name|value|value_doubled|
+---+--------+-----+-------------+
|100|Name_100| 1000|         2000|
| 99| Name_99|  990|         1980|
| 98| Name_98|  980|         1960|
| 97| Name_97|  970|         1940|
| 96| Name_96|  960|         1920|
+---+--------+-----+-------------+
only showing top 5 rows


✅ Benefits:
   • Clean and readable
   • No intermediate variables
   • Easy to understand flow
   • Professional code style


---

## PART 7: PRACTICAL EXAMPLES

In [54]:
# Create realistic employee dataset
from faker import Faker
import random

fake = Faker()
Faker.seed(42)
random.seed(42)

employee_data = []
for i in range(1, 1001):
    employee_data.append({
        "id": i,
        "name": fake.name(),
        "age": random.randint(22, 65),
        "department": random.choice(["Engineering", "Sales", "HR", "Marketing"]),
        "salary": random.randint(40000, 150000)
    })

df_emp = spark.createDataFrame(employee_data)
print(f"✅ Created employee dataset: {df_emp.count()} rows")

✅ Created employee dataset: 1000 rows


### Example 1: Find high earners in Engineering

In [55]:
print("📌 Example 1: High earners in Engineering (salary > 100k)")

result = df_emp \
    .filter(col("department") == "Engineering") \
    .filter(col("salary") > 100000) \
    .select("name", "age", "salary") \
    .orderBy(col("salary").desc())

print(f"\nFound {result.count()} high earners")
result.show(10)

📌 Example 1: High earners in Engineering (salary > 100k)

Found 102 high earners
+---------------+---+------+
|           name|age|salary|
+---------------+---+------+
|Margaret Harper| 33|149602|
|   Jose Schultz| 31|149392|
|  Tammie Bright| 52|147756|
| Nicholas Payne| 40|147343|
|   Sherry Tapia| 24|147216|
|   Joseph Smith| 29|146099|
|      Glen Wood| 38|145464|
|  Sara Johnston| 41|144363|
|Casey Hernandez| 34|143692|
|  Julie Herrera| 60|143445|
+---------------+---+------+
only showing top 10 rows



### Example 2: Department statistics

In [56]:
print("📌 Example 2: Department statistics")

result = df_emp \
    .groupBy("department") \
    .agg(
        count('name').alias("employee_count"),
        avg("salary").alias("avg_salary"),
        min("salary").alias("min_salary"),
        max("salary").alias("max_salary")
    ) \
    .orderBy(col("avg_salary").desc())

result.show()

📌 Example 2: Department statistics


TypeError: 'int' object is not callable

### Example 3: Complex query with multiple transformations

In [57]:
print("📌 Example 3: Senior employees (age > 40) with above-average salary")

# Calculate average salary
avg_salary = df_emp.agg(avg("salary")).collect()[0][0]
print(f"   Average salary: ${avg_salary:,.2f}")

# Find senior employees with above-average salary
result = df_emp \
    .filter(col("age") > 40) \
    .filter(col("salary") > avg_salary) \
    .withColumn("salary_vs_avg", col("salary") - avg_salary) \
    .select("name", "age", "department", "salary", "salary_vs_avg") \
    .orderBy(col("salary_vs_avg").desc())

print(f"\nFound {result.count()} senior high earners")
result.show(10)

📌 Example 3: Senior employees (age > 40) with above-average salary
   Average salary: $94,912.75

Found 288 senior high earners
+--------------------+---+----------+------+-----------------+
|                name|age|department|salary|    salary_vs_avg|
+--------------------+---+----------+------+-----------------+
|    Curtis Wilkerson| 49|     Sales|149926|55013.25199999999|
|Elizabeth Oliver DDS| 48|        HR|149433|54520.25199999999|
|       Curtis Taylor| 61| Marketing|149055|54142.25199999999|
|        Gregory Peck| 50|     Sales|148926|54013.25199999999|
|        Karen Graham| 50| Marketing|148864|53951.25199999999|
|        Ashley Hicks| 54|        HR|148851|53938.25199999999|
|    Matthew Schwartz| 43|     Sales|148840|53927.25199999999|
|        Derek Zuniga| 57|        HR|148711|53798.25199999999|
|       Tracey Wagner| 61| Marketing|148395|53482.25199999999|
|         Mary Miller| 57|        HR|148175|53262.25199999999|
+--------------------+---+----------+------+---------

---

## ✅ SUMMARY

### 📚 What you learned:

**1️⃣ Transformations vs Actions:**
- Transformations: Lazy, return DataFrame
- Actions: Eager, trigger computation

**2️⃣ Lazy Evaluation:**
- Transformations build execution plan
- Actions trigger execution
- Spark optimizes entire plan before execution

**3️⃣ Narrow vs Wide Transformations:**
- Narrow: No shuffle (fast) - filter, select, map
- Wide: Shuffle required (slow) - groupBy, join, orderBy

**4️⃣ Execution Plans:**
- Read from bottom to top
- Understand optimization steps

**5️⃣ Best Practices:**
- Chain operations for readability
- Use cache() for multiple actions
- Minimize wide transformations
- Always unpersist() when done

### 🎯 Key Takeaways:
- ✅ Understand when computation happens
- ✅ Minimize wide transformations
- ✅ Use caching for repeated actions
- ✅ Chain operations for clarity
- ✅ Read execution plans to optimize

### 📝 Next: Partitioning & Caching Deep Dive